In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
vistas = pd.read_csv('../data/adri_data/entrenamiento_vistas.csv');
vistas.head()

,idaviso,timestamp,idpostulante,valor_educacion,sexo,edad,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,...,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1112400775,5.857419e+06,kPjr2K0,9.0,1,20,0,0,0,0,...,1,1,0,0,0,0,0,0,182,489
1,1112400775,6.041136e+06,rmRpqEA,12.0,1,32,0,0,0,0,...,1,1,0,0,0,0,0,0,182,489
2,1112400775,5.856896e+06,EzEovD2,4.0,1,22,0,0,0,0,...,1,1,0,0,0,0,0,0,182,489
3,1112400775,5.306593e+06,jkdBJNb,6.0,1,21,0,0,0,0,...,1,1,0,0,0,0,0,0,182,489
4,1112400775,6.781914e+06,amr6kq,11.0,2,27,0,0,0,0,...,1,1,0,0,0,0,0,0,182,489


In [3]:
postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones.csv');
postulaciones.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad
0,1111556097,0,0,0,0,0,0,0,0,0,...,0,0,0,50,0,pzeVW63,7.892653e+06,17.0,1,44
1,1112370711,1,0,0,0,0,0,0,0,0,...,0,0,0,143,11,pzeVW63,8.331531e+06,17.0,1,44
2,1112326655,1,0,0,0,0,0,0,0,0,...,0,0,0,29,28,pzeVW63,1.066392e+07,17.0,1,44
3,1112350182,1,1,0,0,0,0,0,0,0,...,0,0,0,119,33,pzeVW63,9.711031e+06,17.0,1,44
4,1112383837,1,0,0,0,0,0,0,0,0,...,0,0,0,115,41,pzeVW63,8.418549e+06,17.0,1,44


In [4]:
postulaciones.columns

Index(['idaviso', 'esta_online', 'titulo_analista', 'titulo_tecnico',
       'titulo_administrativo', 'titulo_vendedor', 'titulo_comercial',
       'titulo_ventas', 'titulo_sr', 'titulo_importante', 'titulo_jefe',
       'titulo_empresa', 'titulo_asistente', 'titulo_supervisor', 'titulo_jr',
       'titulo_contable', 'desc_experiencia', 'desc_empresa', 'desc_trabajo',
       'desc_excluyente', 'desc_manejo', 'desc_importante', 'desc_equipo',
       'desc_tareas', 'desc_cliente', 'desc_clientes', 'desc_conocimientos',
       'idzona', 'idciudad', 'idmapacalle', 'idtipo_de_trabajo',
       'idnivel_laboral', 'idnombre_area', 'iddenominacion_empresa',
       'idpostulante', 'timestamp', 'valor_educacion', 'sexo', 'edad'],
      dtype='object')

In [5]:
vistas = vistas[postulaciones.columns]

In [6]:
postulaciones = postulaciones[postulaciones.columns]

In [7]:
vistas['se_postulo'] = 0

In [8]:
postulaciones['se_postulo'] = 1

In [9]:
vistas.shape

(5525669, 40)

In [10]:
postulaciones.shape

(6188634, 40)

In [11]:
vistas_y_postulaciones = pd.concat([postulaciones, vistas], axis=0)
vistas_y_postulaciones.shape

(11714303, 40)

# Agrego la tercer clase

In [12]:
top100 = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
top100.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [13]:
top100_full = pd.merge(top100, vistas_y_postulaciones, how='left', on=['idaviso', 'idpostulante'])
top100_full.head()

,id,idaviso,idpostulante,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,...,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,timestamp,valor_educacion,sexo,edad,se_postulo
0,0,739260,6M9ZQR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,739260,6v1xdL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,739260,ezRKm9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,758580,1Q35ej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,758580,EAN4J6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
excluded = top100_full[top100_full['esta_online'].isnull()]
excluded.shape

(60066, 41)

In [15]:
excluded = excluded[['id', 'idaviso', 'idpostulante']]
excluded.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [16]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalles.csv', encoding='utf-8')
avisos_detalles.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,desc_cliente,desc_clientes,desc_conocimientos,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1111556097,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,50,0
1,1112448447,1,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,182,0
2,1112394600,0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,520,0,0,34,0
3,1111266888,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,142,0
4,1112426827,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,114,0


In [17]:
excluded = pd.merge(excluded, avisos_detalles, how='inner', on='idaviso')
excluded.shape

(60066, 36)

In [18]:
postulantes = pd.read_csv('../data/adri_data/postulantes.csv', encoding='utf-8')
postulantes.shape

(504407, 4)

In [19]:
excluded = pd.merge(excluded, postulantes, how='inner', on='idpostulante')
excluded.shape

(60066, 39)

In [20]:
excluded['se_postulo'] = 0

In [21]:
excluded.shape

(60066, 40)

In [22]:
timestamps = pd.DataFrame(vistas_y_postulaciones.groupby('idaviso')['timestamp'].mean()).reset_index()
timestamps.head()

,idaviso,timestamp
0,820850,4.968944e+06
1,1112360,4.996644e+06
2,1141070,4.309782e+06
3,1144960,4.140725e+06
4,1145170,4.565508e+06


In [23]:
excluded = pd.merge(excluded, timestamps, on='idaviso', how='left')
excluded.shape

(60066, 41)

In [24]:
excluded = excluded[vistas_y_postulaciones.columns]
excluded.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo
0,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,6M9ZQR,NaN,21.0,1,42,-1
1,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,6v1xdL,NaN,11.0,2,30,-1
2,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,ezRKm9,NaN,12.0,1,36,-1
3,758580,0,0,0,0,0,0,0,1,0,...,0,3,168,3016,1Q35ej,NaN,17.0,2,68,-1
4,758580,0,0,0,0,0,0,0,1,0,...,0,3,168,3016,EAN4J6,NaN,10.0,1,32,-1


In [25]:
timestampmean = vistas_y_postulaciones['timestamp'].mean()
timestampmean

7412455.941485848

In [26]:
excluded.loc[excluded['timestamp'].isnull(), 'timestamp'] = timestampmean
excluded.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo
0,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,6M9ZQR,7.412456e+06,21.0,1,42,-1
1,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,6v1xdL,7.412456e+06,11.0,2,30,-1
2,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,ezRKm9,7.412456e+06,12.0,1,36,-1
3,758580,0,0,0,0,0,0,0,1,0,...,0,3,168,3016,1Q35ej,7.412456e+06,17.0,2,68,-1
4,758580,0,0,0,0,0,0,0,1,0,...,0,3,168,3016,EAN4J6,7.412456e+06,10.0,1,32,-1


In [27]:
excluded[excluded['timestamp'].isnull()].head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo


In [28]:
excluded.loc[excluded['timestamp'].isnull(), 'timestamp'] = timestampmean
excluded.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo
0,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,6M9ZQR,7.412456e+06,21.0,1,42,-1
1,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,6v1xdL,7.412456e+06,11.0,2,30,-1
2,739260,0,0,0,0,0,1,0,0,0,...,0,2,29,854,ezRKm9,7.412456e+06,12.0,1,36,-1
3,758580,0,0,0,0,0,0,0,1,0,...,0,3,168,3016,1Q35ej,7.412456e+06,17.0,2,68,-1
4,758580,0,0,0,0,0,0,0,1,0,...,0,3,168,3016,EAN4J6,7.412456e+06,10.0,1,32,-1


# Mergeo con la tercer clase

In [29]:
vistas_y_postulaciones = pd.concat([vistas_y_postulaciones, excluded], axis=0)
vistas_y_postulaciones.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo
0,1111556097,0,0,0,0,0,0,0,0,0,...,0,0,50,0,pzeVW63,7.892653e+06,17.0,1,44,1
1,1112370711,1,0,0,0,0,0,0,0,0,...,0,0,143,11,pzeVW63,8.331531e+06,17.0,1,44,1
2,1112326655,1,0,0,0,0,0,0,0,0,...,0,0,29,28,pzeVW63,1.066392e+07,17.0,1,44,1
3,1112350182,1,1,0,0,0,0,0,0,0,...,0,0,119,33,pzeVW63,9.711031e+06,17.0,1,44,1
4,1112383837,1,0,0,0,0,0,0,0,0,...,0,0,115,41,pzeVW63,8.418549e+06,17.0,1,44,1


In [30]:
vistas_y_postulaciones.shape

(11774369, 40)

# Creo la lista de Ids numericos para los postulantes

In [31]:
# Convierto a numerico el id postulante
lb_make = LabelEncoder()
vistas_y_postulaciones["idpostulante_int"] = lb_make.fit_transform(vistas_y_postulaciones["idpostulante"])
vistas_y_postulaciones[["idpostulante", "idpostulante_int"]].head()

,idpostulante,idpostulante_int
0,pzeVW63,417165
1,pzeVW63,417165
2,pzeVW63,417165
3,pzeVW63,417165
4,pzeVW63,417165


In [32]:
# postulantes_ids = vistas_y_postulaciones[["idpostulante", "idpostulante2"]].drop_duplicates()
# postulantes_ids.to_csv('../data/adri_data/entrenamiento_id_postulantes.csv', encoding='utf-8', index=False)

In [33]:
# vistas_y_postulaciones.drop('idpostulante', axis=1, inplace=True)
# vistas_y_postulaciones.rename({'idpostulante_int': 'idpostulante'}, inplace=True, axis=1)
# vistas_y_postulaciones.head()

In [34]:
# vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8', index=False)

# Luego de mergear

In [35]:
# vistas_y_postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8')

In [36]:
vistas_y_postulaciones.shape

(11774369, 41)

In [37]:
vistas_y_postulaciones.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo,idpostulante_int
0,1111556097,0,0,0,0,0,0,0,0,0,...,0,50,0,pzeVW63,7.892653e+06,17.0,1,44,1,417165
1,1112370711,1,0,0,0,0,0,0,0,0,...,0,143,11,pzeVW63,8.331531e+06,17.0,1,44,1,417165
2,1112326655,1,0,0,0,0,0,0,0,0,...,0,29,28,pzeVW63,1.066392e+07,17.0,1,44,1,417165
3,1112350182,1,1,0,0,0,0,0,0,0,...,0,119,33,pzeVW63,9.711031e+06,17.0,1,44,1,417165
4,1112383837,1,0,0,0,0,0,0,0,0,...,0,115,41,pzeVW63,8.418549e+06,17.0,1,44,1,417165


In [38]:
dup = vistas_y_postulaciones.duplicated(['idaviso', 'idpostulante'])
vistas_y_postulaciones[dup].shape

(1774195, 41)

In [39]:
vistas_y_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], inplace=True)
vistas_y_postulaciones.shape

(10000174, 41)

In [40]:
vistas_y_postulaciones = vistas_y_postulaciones.sample(frac=1).reset_index(drop=True)

In [41]:
vistas_y_postulaciones.shape

(10000174, 41)

In [42]:
vistas_y_postulaciones.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo,idpostulante_int
0,1112378604,1,0,0,0,0,0,0,0,0,...,0,142,23,owaL4oe,8.258221e+06,10.0,1,26,1,407299
1,1112403640,0,0,0,0,0,1,0,0,0,...,0,172,4119,QNr8KP4,7.558382e+06,10.0,-1,-1,1,246518
2,1111714909,0,0,0,0,0,0,0,0,0,...,0,142,439,4re9XKe,9.424259e+06,6.0,1,33,0,49684
3,1112364754,1,0,0,1,0,0,0,0,0,...,0,2,334,3NPL0ar,9.038178e+06,6.0,1,31,0,38778
4,1112432367,1,0,0,0,1,0,0,0,0,...,0,182,5,kPj23x0,6.087481e+06,10.0,1,21,1,381659


In [43]:
vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas5.csv', encoding='utf-8', index=False)